In [0]:
from datetime import datetime
import pandas as pd
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("WebActivityAnalytics").getOrCreate()

# Generate sample data
data = {
    'user_id': [1, 2, 1, 3, 2],
    'timestamp': [datetime(2022, 1, 1), datetime(2022, 1, 1), datetime(2022, 1, 2), datetime(2022, 1, 2), datetime(2022, 1, 3)],
    'page_visited': ['home', 'product', 'home', 'product', 'contact']
}

df = spark.createDataFrame(pd.DataFrame(data))

# Save data to Parquet format for persistence
df.write.mode('overwrite').parquet("/mnt/databricks/sample_data.parquet")


In [0]:
# Show the first few rows of the DataFrame
df.show()

# Display summary statistics
df.describe().show()

# Check for missing values
df.na.drop().show()

# Check for duplicate records
df.dropDuplicates().show()


+-------+-------------------+------------+
|user_id|          timestamp|page_visited|
+-------+-------------------+------------+
|      1|2022-01-01 00:00:00|        home|
|      2|2022-01-01 00:00:00|     product|
|      1|2022-01-02 00:00:00|        home|
|      3|2022-01-02 00:00:00|     product|
|      2|2022-01-03 00:00:00|     contact|
+-------+-------------------+------------+

+-------+------------------+------------+
|summary|           user_id|page_visited|
+-------+------------------+------------+
|  count|                 5|           5|
|   mean|               1.8|        NULL|
| stddev|0.8366600265340756|        NULL|
|    min|                 1|     contact|
|    max|                 3|     product|
+-------+------------------+------------+

+-------+-------------------+------------+
|user_id|          timestamp|page_visited|
+-------+-------------------+------------+
|      1|2022-01-01 00:00:00|        home|
|      2|2022-01-01 00:00:00|     product|
|      1|2022-01-0

In [0]:
# Optimize schema by choosing appropriate data types
df = df.withColumn("user_id", df["user_id"].cast("integer"))

# Optimize storage and retrieval by partitioning
df.write.mode('overwrite').partitionBy("timestamp").parquet("/mnt/databricks/partitioned_data.parquet")

# Cache the DataFrame
df.cache()

# Perform operations on the cached DataFrame
df_filtered = df.filter(df['page_visited'] == 'home')

# Create a small DataFrame to be broadcasted
small_df = df.select('user_id').distinct()

# Perform a broadcast join
df_joined = df.join(broadcast(small_df), 'user_id', 'inner')


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2165825361569193>, line 17
     14 small_df = df.select('user_id').distinct()
     16 # Perform a broadcast join
---> 17 df_joined = df.join(broadcast(small_df), 'user_id', 'inner')

NameError: name 'broadcast' is not defined

In [0]:
# Aggregate user activity metrics
activity_metrics = df.groupBy('user_id').agg(count('page_visited').alias('page_views'))

# Perform time-based analysis
from pyspark.sql.functions import hour
time_analysis = df.groupBy(hour('timestamp').alias('hour')).agg(count('*').alias('activity_count'))


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2165825361569194>, line 2
      1 # Aggregate user activity metrics
----> 2 activity_metrics = df.groupBy('user_id').agg(count('page_visited').alias('page_views'))
      4 # Perform time-based analysis
      5 from pyspark.sql.functions import hour

NameError: name 'count' is not defined

In [0]:
import matplotlib.pyplot as plt

# Create a bar chart using Matplotlib
activity_metrics_pd = activity_metrics.toPandas()
plt.bar(activity_metrics_pd['user_id'], activity_metrics_pd['page_views'])
plt.show()


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2165825361569195>, line 4
      1 import matplotlib.pyplot as plt
      3 # Create a bar chart using Matplotlib
----> 4 activity_metrics_pd = activity_metrics.toPandas()
      5 plt.bar(activity_metrics_pd['user_id'], activity_metrics_pd['page_views'])
      6 plt.show()

NameError: name 'activity_metrics' is not defined